In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
dt=pd.read_csv('../input/dta3.csv')


In [ ]:
dt=pd.get_dummies(dt)

In [ ]:
dt=dt.drop(columns=['B_ELECTRONIC_AUCTION_N','PROCEDURE_RES','SECTOR_na','B_GPA_N','TYPE_OF_CONTRACT_W','CAE_TYPE_Z','AUTHORITY_COUNTRY_UK'] )

In [ ]:
print(dt.columns)
len(dt)

In [ ]:
dt = dt[pd.notnull(dt['AWARD_VALUE_EURO'])]
dt = dt[dt['AWARD_VALUE_EURO'] != 0]
len(dt)



In [ ]:
dt = dt[pd.notnull(dt['AWARD_EST_VALUE_EURO'])]
dt = dt[dt['AWARD_EST_VALUE_EURO'] != 0]
len(dt)



In [ ]:
dt = dt[pd.notnull(dt['NUMBER_OFFERS'])]
dt = dt[dt['NUMBER_OFFERS'] != 0]
len(dt)

In [ ]:
import numpy as np
dt['LNRATIO']=(np.log(dt['AWARD_VALUE_EURO']/dt['AWARD_EST_VALUE_EURO']))
len(dt)


In [ ]:
#log of 'df_train['month'] = pd.to_datetime(df_train.Date).dt.month
dt['LN_AWARD_VALUE_EURO'] = np.log(dt['AWARD_VALUE_EURO'])
dt['LN_AWARD_EST_VALUE_EURO'] = np.log(dt['AWARD_EST_VALUE_EURO'])

In [ ]:
dt2=dt

In [ ]:
dt = dt[(dt.LNRATIO>-1.386) & (dt.LNRATIO<0.642)]


In [ ]:
train=dt[(dt.YEAR<2017)]
test=dt[(dt.YEAR==2017)]

In [ ]:
y_train = train['LNRATIO']
X_train = train.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO', 'LNRATIO', 'LN_AWARD_VALUE_EURO'], axis=1)
y_test = test['LNRATIO']
X_test = test.drop(['AWARD_VALUE_EURO', 'AWARD_EST_VALUE_EURO',  'LNRATIO' ,'LN_AWARD_VALUE_EURO'], axis=1)


In [ ]:
import math
from scipy.stats import spearmanr
def rmse(x,y): return math.sqrt(((x-y)**2).mean())
def print_score(m):
    print('Train rmse '+str(rmse(m.predict(X_train), y_train))) 
    print('Test  rmse '+str(rmse(m.predict(X_test), y_test)))
    print('Train '+str(spearmanr(m.predict(X_train),y_train)))
    print('Test  '+str(spearmanr(m.predict(X_test),y_test)))

In [ ]:
import numpy as np
import pandas as pd

from hyperopt import fmin, hp, tpe, Trials, space_eval
from hyperopt.fmin import fmin

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer

import xgboost as xgb

import lightgbm as lgbm

In [ ]:
def obj(params,X,y):
    params = {
        'max_depth': int(params['max_depth']),
        'gamma': "{:.3f}".format(params['gamma']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
    }
    
    clf = xgb.XGBRegressor(
        n_estimators=250,
        learning_rate=0.05,
        n_jobs=4,
        **params
    )
    
    score = cross_val_score(clf, X, y, scoring='neg_mean_squared_error',cv=2).mean()
    print("Gini {:.3f} params {}".format(score, params))
    return score
objective = lambda params: (-1)*obj(params, X_train.values, y_train)
spa = {
    'max_depth': hp.choice('max_depth', np.arange(1, 14, dtype=int)),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
}

best = fmin(fn=objective,
            space=spa,
            algo=tpe.suggest,
            max_evals=50)

In [ ]:
bestXGB = space_eval(spa, best)
print("BEST PARAMETERS: " + str(bestXGB))

In [ ]:

from sklearn.metrics import r2_score
est = xgb.XGBRegressor(**bestXGB) 
# Fit    
est.fit(X_train.values, y_train)
y_pred = est.predict(X_test.values)
 
# Predict
score = rmse(y_test, y_pred)

print("RMSE SCORE ON TEST DATA: {}".format(score))
r2a = r2_score(y_test, y_pred)
print("RMSE SCORE ON TEST DATA: {}".format(score))